# 基于asyncio包的并发

asyncio包使用事件循环驱动的协程实现并发。这个包也确定了协程以及future是符合Python风格的异步代码编写方式。

本章以服务器搭建以及并发下载为例介绍基于asyncio的并发，但是相关内容和我目前的项目方向相差甚远，因此主要是总结下重要内容，实践以及实现细节将后续补充

## 线程和协程

协程相较于线程更“轻量”。在Python中，多线程可以由threading实现，基于协程的并发则可以使用asyncio实现。

本书用一个例子的两种实现描述了线程和协程的差异以及asyncio包的一些注意事项。

主要区别如下

1. 基于threading的线程需要创Thread对象，基于asyncio的协程则需要创建Task对象。这两者从功能上来看是等效的
2. Task用于驱动协程，Thread用于调用可调用对象。代码可以明显反映这一区别 —— 传入Task对象的是spin("thinking!")，而传入Thread对象的是spin
3. Task对象并不用手动实例化
4. Task在创建时已经排定了协程的运行时间，Thread则需要显式的指定运行（Thread_obj.start()）
5. 协程和线程的暂停方式不同。具体来说，线程无法利用API从外部终止，但是协程可以主动调用Task_obj.cancel()方法在协程内部抛出CancelledError迫使协程终止
6. 基于asyncio包的协程必须在主线程中由事件循环执行

### asyncio.Future和concurrent.futures.Future
asyncio.Future和concurrent.futures.Future接口基本一致，但是功能不同，不可以互换。

asyncio.Future和concurrent.futures.Future类似，该类也提供.done()、.add_done_callback(...)以及.result()等方法。

其中，.result()方法的行为在两个类中完全不一样。asyncio.Future中的.result()方法不支持设置超时时间，此外调用该方法时若协程还没运行完毕，.result()也不会等待结果，而是直接抛出asyncio.InvalidStateError

asyncio.Future主要和yield from配合使用，而yield from完全可以实现.add_done_callback(...)以及.result()的功能，因此这两个方法也较少使用

### 从future、任务以及协程中产出

asyncio中future和协程关系密切。若使用Furues进行并发，需要手动处理并发操作并使用接受结果，但是在asyncio中则可以配合yield from实现非常轻松的并发以及结果接收。

与基本的协程一样，asyncio也要求排定协程的运行时间，此外asyncio还要求使用Task对象包装协程。

创建Task对象的方法有很多，例如使用BaseEventLoop.create_task创建，或者使用asyncio.ensure_future创建。这两种方法均是创建一个Task，若有多个任务则可以使用asyncio.wait(...)创建，.wait(...)要求传入由future或者协程构成的可迭代对象并将这些对象变为Future类的实例。asyncio.wait可以设定timeout以及return_when，这两个参数在一定程度上缓解了asyncio.Future.result()无法设置超时时间的问题

创建Task后，asyncio使用事件循环驱动所有任务/协程。事件循环的返回值有两个，一个是已完成的future的结果，另一个则记录了没有完成的future。

### yield from

在asyncio的框架上使用yield from符合16章中描述的yield from用法特点：

1. yield from连接的多个协程最终必须由一个不是协程的调用方驱动，即调用方显式/隐式的调用next以及send方法
2. 整个协程的最内层应当是一个简单的生成器或者可迭代对象

第一条对应着asyncio的事件循环；第二条，实际上asyncio会调用包中的某个协程函数/方法或者其他包中实现类似功能的协程。

综上，asyncio实际上是构建了一个“管道”：事件循环以及各种异步I/O函数之间的“管道”

## 避免阻塞型调用

相较于其他操作，执行硬盘或者网络I/O操作需要耗费大量的时间，因此应当尽量避免阻塞型调用中止主进程。实现这一目标有两种方法：
1. 并行，直接在单独的线程中运行各种阻塞型操作
2. 异步：转换为非阻塞的异步调用

显然asyncio使用的是第二种方法。由于使用了异步I/O，所有的请求将在极短的时间内启动，然后等待响应，当获取到响应后，事件循环会将结果发送给指定的协程并且该协程执行到下一个yield from位置，所有的任务均是异步执行的。

## 从回调到future和协程

回调的嵌套通常出现在相互依赖的操作之间。但是相关实现难以阅读并且难以维护，此外，还可能需要依赖闭包或者全局的数据结构来保存上下文数据。

协程显然为上述的过程提供了有效的解决方法，基于上一章以及本章上述小结的讨论，通过yield from构建通道能够很方便的组织多个协程。此外，yield from还能够非常合理的处理嵌套的回调。嵌套的回调实际上就是顺序执行的多个异步操作，对于协程，完全可以在一个函数中顺序执行多次yield，以实现多次异步操作。

基于协程实现多个异步操作的顺序执行还能非常方便的进行debug —— 上下文均得到了保存并且完全可以使用try/except处理协程抛出的错误。

## 总结

1. asyncio包为Python实现并发编程提供了全新方法，其核心就是利用asyncio事件循环驱动多个协程
2. 相较于线程，协程更轻量。此外，协程在用户态中运行
3. asyncio对yield from的应用符合yield from的特点：yield from连接的多个协程由一个不是协程的调用方驱动、其最内层是一个简单的生成器或者可迭代对象
4. 执行硬盘或者网络I/O操作需要耗费大量的时间，因此应当避免阻塞型调用，具体而言有两种方法：并行、异步。asyncio显然是依靠异步实现并发
5. 回调是一种低层的概念，多个回调嵌套会造成“回调地狱”。不同于回调，协程能够非常自然的解决多个异步操作顺序执行的问题 —— 在一个协程中执行多次yield。相较于回调，利用协程处理多个异步操作顺序执行的问题能够非常方便的进行debug 